In [41]:
from random import randrange, random
from time import strptime, mktime
import requests

idRand = None                    # ідентефікатор аккаунта
account = None      

N = 624                          # що це і звідки ??
M = 397                          # що це і звідки??

MATRIX_A = 0x9908B0DF            # що це і звідки ??
UPPER_MASK = 0x80000000          # що це і звідки ??
LOWER_MASK = 0X7FFFFFFF          # що це і звідки ??
DEFAULT_SEED = 5489              # що це і звідки ??

mt = [0 for i in range(N + 1)]   # масив (чому mt?) для чого він нам?
mtiSize = N + 1                      # розмір масиву

modulus = 2 ** 32                # константа ???
a = c = 0                        # робоча константа


def tempering(y):                # що робить цей метод??? чого там такі числа
    y ^= y >> 11                 
    y ^= y << 7  & 0x9d2c5680
    y ^= y << 15 & 0xefc60000
    y ^= y >> 18
    return y


def unTempering(y):               # що робить цей метод??? чога там такі числа
    y ^= y >> 18
    y ^= y << 15 & 0xefc60000
    y ^= y << 7  & 0x9d2c5680
    y ^= y << 14 & 0x94284000
    y ^= y << 21 & 0x14200000
    y ^= y << 28 & 0x10000000
    y ^= y >> 11 ^ y >> 22
    return y


# генератор який геренує масив і розмір з одного значення і ініціалізує mt і mtiSize

def init_genrand_1(s=DEFAULT_SEED): 
    global mt, mtiSize
    ustates = [s & 0xffffffff]
    for i in range(1, N):
        ustates.append((1812433253 * (ustates[i - 1] ^
                        ustates[i - 1] >> 30) + i) & 0xffffffff)
    mt = ustates
    mtiSize = N # чому тут N ?????

# генератор який геренує масив і розмір з масиву і ініціалізує mt і mtiSize
# unTempering? ?????
def init_genrand_n(states):
    global mt, mtiSize
    ustates = []
    for state in states:
        ustates.append(unTempering(state))
    mt = ustates
    mtiSize = N # чому тут N ?????

# функція яка вираховує зміну y
# для чого вона і звідки всі ці формули?
def genrand_int32():
    global mt, mtiSize
    y = 0
    mag01 = [0, MATRIX_A]
    if mtiSize >= N:
        if mtiSize == N + 1:
            init_genrand_1()
        for kk in range(N - M):
            y = (mt[kk] & UPPER_MASK) | (mt[kk + 1] & LOWER_MASK)
            mt[kk] = mt[kk + M] ^ (y >> 1) ^ mag01[y & 0x1]
        for kk in range(kk, N - 1):
            y = (mt[kk] & UPPER_MASK) | (mt[kk + 1] & LOWER_MASK)
            mt[kk] = mt[kk - 227] ^ (y >> 1) ^ mag01[y & 0x1]
        y = (mt[N - 1] & UPPER_MASK) | (mt[0] & LOWER_MASK)
        mt[N - 1] = mt[M - 1] ^ (y >> 1) ^ mag01[y & 0x1]
        mtiSize = 0
    y = mt[mtiSize]
    mtiSize += 1
    y = tempering(y)
    return y


def findVariables(numbers):
    global a, c
    diff1 = numbers[2] - numbers[1]
    diff2 = numbers[1] - numbers[0]
    inverse = pow(diff2, modulus - 2, modulus) # остача від (diff2  в степені (modulus - 2)) поділити на modulus

    a = diff1 * inverse % modulus
    c = (numbers[1] - (a * numbers[0])) % modulus


def lcg_next(last):
    last = (a * last + c) % modulus
    return last


def LcgCrack():
    global mt, mtiSize
#    mt = [0 for i in range(N + 1)] 
#    mtiSize = N + 1

    numbers = []
    print('Далі буде три пари "account.money/realNumber"')
    for i in range(3):
        numbers.append(makeBet("Lcg", 1, 1))

    findVariables(numbers)
    print(f"\na = {a}")
    print(f"c = {c}")

    next0 = lcg_next(numbers[2])
    print(end=f"\nПередбачене наступне число = {next0}")
    if not makeBet("Lcg", 500, next0, print_mode='full_print'): return

    next1 = lcg_next(next0)
    print(end=f"\nПередбачене наступне число = {next1}")
    makeBet("Lcg", 501, next1, print_mode='full_print')


def MTCrack():
    global mt, mtiSize
#    mt = [0 for i in range(N + 1)] 
#    mtiSize = N + 1

    Time = account['deletionTime']
    number = makeBet("Mt", 1, 1, print_mode='full_print')
    print(f'{number = }')
    seed = int(mktime(strptime(Time.rsplit('.', 1)[0], '%Y-%m-%dT%H:%M:%S'))) - 3600
    predictedNumber = 0
    i = 0
    print(end=f'Далі тисячі спроб до "predictedNumber == number": ')
    while True:
        init_genrand_1(seed)
        predictedNumber = genrand_int32()
        i += 1
        if i % 1000 == 0:
            print(end=f'{i // 1000} ')
        seed += 1
        if predictedNumber == number:
            break
    print('\nДалі буде останніх 10 пар "account.money/realNumber"')
    for i in range(10):
        makeBet("Mt", 110, genrand_int32())


def BetterMTCrack():
    global mt, mtiSize, i
#    mt = [0 for i in range(N + 1)] 
#    mtiSize = N + 1

 #    numbers = {}
    numbers = []

#    print(f'Далі буде {N} пари "account.money/realNumber"')
    print(end=f'Далі сотні спроб до "{N}": ')
    for i in range(N):
#        numbers[i] = makeBet("BetterMt", 1, 1, print_mode='i')
        numbers.append(makeBet("BetterMt", 1, 1, print_mode='i'))

    init_genrand_n(numbers)

    print('\nДалі буде останніх 10 пар "account.money/realNumber"')
    for i in range(10):
        predictedNumber = genrand_int32()
        makeBet("BetterMt", 110, predictedNumber)


def createAccount():
    global idRand, account
    for i in range(10):
        idRand = randrange(30000) 
        response = requests.get('http://95.217.177.249/casino/createacc',
                                        params=dict(id=idRand))
        account = response.json()
        print(account)
        if "error" not in account:
            break
    else:
        idRand = None
        account = None

def makeBet(mode, betAmount, number, print_mode='par_print'):
    response = requests.get(f'http://95.217.177.249/casino/play{mode}',
                        params=dict(id=idRand, bet=betAmount, number=number))
    bet = response.json()
    if print_mode == 'full_print' or \
            'account' not in bet or \
            'realNumber' not in bet:
        print(f'\n{bet}')
    elif print_mode == 'par_print':
        print(end=f'{bet["account"]["money"]}/{bet.get("realNumber")} ')
    elif i % 100 == 99:
        print(end=f'{(1 + i) // 100} ')
    return bet.get('realNumber')

In [42]:
mt = [0 for i in range(N + 1)] 
mtiSize = N + 1
createAccount()

{'id': '16156', 'money': 1000, 'deletionTime': '2021-12-26T17:35:36.2377333Z'}


In [43]:
MTCrack()


{'message': 'You lost this time', 'account': {'id': '16156', 'money': 999, 'deletionTime': '2021-12-26T17:35:36.2377333Z'}, 'realNumber': 3872473991}
number = 3872473991
Далі тисячі спроб до "predictedNumber == number": 1 2 3 4 5 6 7 
Далі буде останніх 10 пар "account.money/realNumber"
110889/3565289673 220779/4254317374 330669/2865293203 440559/2413961621 550449/4232075790 660339/3026978256 770229/2553133758 880119/685159090 990009/4031205636 1099899/3711940558 

In [44]:
BetterMTCrack()

Далі сотні спроб до "624": 1 2 3 4 5 6 
Далі буде останніх 10 пар "account.money/realNumber"
1099165/980179899 1099055/573764481 1098945/1257158049 1098835/961336597 1098725/1749949723 1098615/3235813013 1098505/3562570789 1098395/4166575207 1098285/1045719632 1098175/860275442 

In [45]:
LcgCrack()
LcgCrack()
LcgCrack()
LcgCrack()
LcgCrack()
LcgCrack()
LcgCrack()
LcgCrack()

Далі буде три пари "account.money/realNumber"
1098174/744380949 1098173/-517282704 1098172/294726927 
a = 144289207
c = 3897249389

Передбачене наступне число = 4168918822
{'type': 'https://tools.ietf.org/html/rfc7231#section-6.5.1', 'title': 'One or more validation errors occurred.', 'status': 400, 'traceId': '|9c33b892-4f1a57ad4f8c317b.', 'errors': {'number': ["The value '4168918822' is not valid."]}}
Далі буде три пари "account.money/realNumber"
1098171/1597585186 1098170/1084247065 1098169/-782907740 
a = 1467699227
c = 1225186179

Передбачене наступне число = 1475481295
{'message': 'You lost this time', 'account': {'id': '16156', 'money': 1097669, 'deletionTime': '2021-12-26T17:35:36.2377333Z'}, 'realNumber': 1894998451}

Передбачене наступне число = 2862881624
{'type': 'https://tools.ietf.org/html/rfc7231#section-6.5.1', 'title': 'One or more validation errors occurred.', 'status': 400, 'traceId': '|9c33b897-4f1a57ad4f8c317b.', 'errors': {'number': ["The value '2862881624' is not